# Importing dependencies and models

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import cv2
import easyocr
import pytesseract

In [2]:
from ultralytics import YOLO
from sort.sort import *

In [5]:
from util import get_car, read_license_plate, write_csv

In [4]:
car_model = YOLO("yolov8n.pt")
license_plate_model = YOLO("./license_detector.pt")
mot_tracker = Sort()
reader = easyocr.Reader(['en'], gpu=False)

Using CPU. Note: This module is much faster with a GPU.


# Steps to make predictions on images

### 1. Read an image

In [6]:
image = cv2.imread("./test images/car-wbs-HR26CH3604_00000.jpeg")

### 2. Detect the license plate 

In [38]:
license_detections = license_plate_model(image)


0: 640x480 (no detections), 323.0ms
Speed: 14.0ms preprocess, 323.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)


In [39]:
for detection in license_detections:
    detection.show()

### 3. Get the position/coordinates of the license plate

In [9]:
for detection in license_detections:
        for box in detection.boxes.data.tolist():
            (x1, y1, x2, y2, cnf, class_id) = box
            

### 4. Crop the original image

In [10]:
license_cropped = image[int(y1): int(y2), int(x1):int(x2), :]

In [11]:
plt.imshow(cv2.cvtColor(license_cropped, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Cropped image (license plate)")
plt.show()

### 5. Convert the cropped image into grayscale

In [12]:
license_gray = cv2.cvtColor(license_cropped, cv2.COLOR_BGR2GRAY)

In [13]:
plt.imshow(cv2.cvtColor(license_gray, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Gray scaled image (license plate)")
plt.show()

### 6. Get the text from license plate using EasyOCR

In [14]:
detections = reader.readtext(license_gray)

In [15]:
if detections:
    license_text = detections[0][1]
    score = detections[0][2]

    print(f"\nDetected Text: {license_text}\nConfidence Score: {score}")



Detected Text: HR 26 CH 3604
Confidence Score: 0.8790917902579659


# Functions to detect and recognize license plates from images

In [31]:
def predict_on_img(image, save, image_name, output_dir):
    """
    Predict the license plate text from the given image and optionally save the annotated image.

    Args:
        image (numpy.ndarray): The input image.
        save (bool): Whether to save the annotated image or not.
        image_name (str): The name of the image stored locally. (Only required when save=True)
        output_dir (str): The directory to save the annotated image.

    Returns:
        None
    """
    # Detecting the license plate from the image
    license_detections = license_plate_model(image)

    # Extracting its coordinates/position
    for detection in license_detections:
        for box in detection.boxes.data.tolist():
            (x1, y1, x2, y2, cnf, class_id) = box
            
    # Displaying the original image
    # plt.imshow(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    # plt.axis("off")
    # plt.title("Original image")
    # plt.show()
    
    # Cropping the original image to get the license plate
    license_cropped = image[int(y1): int(y2), int(x1):int(x2), :]

    # Displaying the cropped image
    # plt.imshow(cv2.cvtColor(license_cropped, cv2.COLOR_BGR2RGB))
    # plt.axis("off")
    # plt.title("Cropped image (license plate)")
    # plt.show()

    # Converting the license plate to gray scale
    license_gray = cv2.cvtColor(license_cropped, cv2.COLOR_BGR2GRAY)

    # plt.imshow(cv2.cvtColor(license_gray, cv2.COLOR_BGR2RGB))
    # plt.axis("off")
    # plt.title("Gray scaled image (license plate)")
    # plt.show()

    # Thresholding --> optional
    ''' _, license_thresh = cv2.threshold(license_gray, threshold, 255, cv2.THRESH_BINARY_INV)

    plt.imshow(cv2.cvtColor(license_thresh, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title("Thresholded image (license plate)")
    plt.show() '''

    # Making detections using the EasyOCR reader
    detections = reader.readtext(license_gray)
    
    if detections:
        license_text = detections[0][1]
        score = detections[0][2]

        print(f"\nDetected Text: {license_text}\nConfidence Score: {score}")

        # Drawing bounding box and annotating text on the original image
        bbox = [(x1, y1), (x2, y1), (x2, y2), (x1, y2)]
        annotated_image = draw_bounding_box(image.copy(), bbox, license_text)

        # Displaying the annotated image
        # plt.imshow(cv2.cvtColor(annotated_image,cv2.COLOR_BGR2RGB))
        # plt.title("Annotated image")
        # plt.axis("off")
        # plt.show()

        if save:
            save_annotated_image(annotated_image, image_name, output_dir)
        
    else:
        print("No text detected")


In [17]:
def draw_bounding_box(image, bbox, text):
    """
    Draw bounding box and text on the image.

    Args:
        image (numpy.ndarray): The original image.
        bbox (list): The bounding box coordinates.
        text (str): The detected text.

    Returns:
        numpy.ndarray: The image with bounding box and text.
    """
    # Convert bounding box to integer coordinates
    bbox = np.array(bbox).astype(int)
    
    # Draw the bounding box
    cv2.polylines(image, [bbox], isClosed=True, color=(0, 255, 0), thickness=2)

    # Get the coordinates for text placement
    x1, y1 = bbox[0]
    x2, y2 = bbox[2]

    # Calculate the width and height of the text box
    (text_width, text_height), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)

    # Draw a filled white rectangle above the bounding box for the text
    text_box_top_left = (x1, y1 - text_height - 10)
    text_box_bottom_right = (x1 + text_width + 10, y1)
    cv2.rectangle(image, text_box_top_left, text_box_bottom_right, (255, 255, 255), cv2.FILLED)

    # Put the detected text on the white rectangle
    text_position = (x1 + 5, y1 - 5)
    cv2.putText(image, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 0), 2)

    return image


In [27]:
def save_annotated_image(annotated_image, image_name, output_dir):
    # Save the annotated image
    output_path = output_dir + "/" + f'{image_name}.jpg'
    success = cv2.imwrite(output_path, annotated_image)
    if success:
        print(f"Annotated image saved to: {output_path}")
    else:
        print(f"Failed to save annotated image to: {output_path}")

In [28]:
OUTPUT_DIR = "./output images"

In [29]:
img = cv2.imread("./test images/9f7c7dce-f9c9-4c95-a053-8f16e6e99466___new_new-maruti-alto-k10-2_625x300_41414071192.jpg.jpeg")

In [34]:
predict_on_img(img, save=False, image_name="abc", output_dir=OUTPUT_DIR)


0: 384x640 1 License_Plate, 328.0ms
Speed: 8.0ms preprocess, 328.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

Detected Text: HR 26 CT 6702
Confidence Score: 0.47553099476656985


In [35]:
os.listdir("./test images")

['03273806-bb1e-48da-8c8b-a0133a90197a___2014-Skoda-Yeti-Test-Drive.jpg.jpeg',
 '07bd977e-d578-49a2-b345-7cee5a4db6bf___new_1031520d1356604430-skoda-rapid-joins-family-edit-sold-wp_000281.jpg.jpeg',
 '0c9ebe94-827d-4c74-9950-6816e70d1bab___IMG_8883.jpeg',
 '15e7c19f-da3c-4f0d-91f3-552afabc8961___41FxrEl7jHL.jpg.jpeg',
 '23d775eb-842f-4f57-ad7a-affafa21660a___215453-new-2016-maruti-ertiga-granite-grey-zdi-shvs-highway-champion-img_8168.jpg.jpeg',
 '254b647f-4cb3-46c3-a206-4e70041f4c78___speedex-number-plates-nandanam-chennai-car-number-plate-dealers-33nu2i6.jpg.jpeg',
 '2adfb5e7-e70a-4749-b5fa-8c233c0d47d9___new_1572920d1478091901-your-favourite-number-plate-font-jetta-german-font.jpg.jpeg',
 '41de2ad0-3785-488e-9f1f-75b76f446cc7___Maruti-Alto-K10-Photos-2.jpg.jpeg',
 '49bdf0d9-4e64-41eb-9c19-eabdc4afb051___Maruti-Suzuki-Ciaz-Photos-30.JPG.jpeg',
 '51d58875-0bdc-4488-84d9-8b7ad5883863___new_Maruti-Suzuki-Ignis-Rear-view-94486.jpg.jpeg',
 '56b46df3-223c-42c8-a6bf-e95e5bac29cc___9214a7543

## Saving the output images into a seperate directory

In [36]:
for image in os.listdir("./test images"):
    img = cv2.imread("./test images/"+image)
    predict_on_img(img,save=True,image_name = image,output_dir=OUTPUT_DIR)


0: 544x640 1 License_Plate, 394.0ms
Speed: 31.0ms preprocess, 394.0ms inference, 3.0ms postprocess per image at shape (1, 3, 544, 640)

Detected Text: HH2Ocs9817
Confidence Score: 0.32078128653423615
Annotated image saved to: ./output images/03273806-bb1e-48da-8c8b-a0133a90197a___2014-Skoda-Yeti-Test-Drive.jpg.jpeg.jpg

0: 480x640 1 License_Plate, 348.0ms
Speed: 10.0ms preprocess, 348.0ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

Detected Text: MH12JC2813
Confidence Score: 0.8381903760315976
Annotated image saved to: ./output images/07bd977e-d578-49a2-b345-7cee5a4db6bf___new_1031520d1356604430-skoda-rapid-joins-family-edit-sold-wp_000281.jpg.jpeg.jpg

0: 448x640 1 License_Plate, 360.0ms
Speed: 8.0ms preprocess, 360.0ms inference, 3.0ms postprocess per image at shape (1, 3, 448, 640)

Detected Text: MH2OBY36 65
Confidence Score: 0.4716799509900124
Annotated image saved to: ./output images/0c9ebe94-827d-4c74-9950-6816e70d1bab___IMG_8883.jpeg.jpg

0: 352x640 1 Lic

UnboundLocalError: cannot access local variable 'y1' where it is not associated with a value

# Detecting and recognizing license plates from video

In [41]:
# class of vehicles present in the dataset used to train YOLO
# ["car","motorcycle","bus","truck"]
vehicles = [2,3,5,7] 

In [42]:
# Initialize video capture (from a file in this case)
cap = cv2.VideoCapture('./test-video.mp4')

# Define the codec and create VideoWriter object
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('output_video.avi', cv2.VideoWriter_fourcc(*'MJPG'), 30, (frame_width, frame_height))

# Initialize SORT tracker
mot_tracker = Sort()

frame_num = 1
while cap.isOpened() and frame_num <= 700:
    ret, frame = cap.read()
    if not ret:
        break
    frame_num = frame_num + 1
    # Detect vehicles
    detections = car_model(frame)[0]  # Replace with your vehicle detection model
    detections_ = []
    for detection in detections.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = detection
        if int(class_id) in vehicles:  # Assuming 'vehicles' is a list of vehicle class IDs
            detections_.append([x1, y1, x2, y2, score])

    # Track vehicles
    track_ids = mot_tracker.update(np.asarray(detections_))

    # Detect license plates
    license_plates = license_plate_model(frame)[0]  # Replace with your license plate detection model
    results = {}
    for license_plate in license_plates.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = license_plate

        # Assign license plate to car
        xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)  # Replace with your function

        if car_id != -1:
            # Crop license plate
            license_plate_crop = frame[int(y1):int(y2), int(x1):int(x2), :]

            # Process license plate
            license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
            _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV)

            # Read license plate number
            license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop_thresh)  # Replace with your OCR function

            if license_plate_text is not None:
                results[car_id] = {
                    'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                    'license_plate': {'bbox': [x1, y1, x2, y2],
                                      'text': license_plate_text,
                                      'bbox_score': score,
                                      'text_score': license_plate_text_score}
                }

    # Draw bounding boxes and overlay text
    for car_id, data in results.items():
        # Draw car bounding box
        car_bbox = data['car']['bbox']
        cv2.rectangle(frame, (int(car_bbox[0]), int(car_bbox[1])), (int(car_bbox[2]), int(car_bbox[3])), (0, 255, 0), 4)

        # Draw license plate bounding box and text
        license_plate_bbox = data['license_plate']['bbox']
        cv2.rectangle(frame, (int(license_plate_bbox[0]), int(license_plate_bbox[1])),
                      (int(license_plate_bbox[2]), int(license_plate_bbox[3])), (0, 0, 255), 4)
        
        # Set font parameters
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 3
        font_thickness = 10

        # Draw text on the license plate
        cv2.putText(frame, data['license_plate']['text'], (int(license_plate_bbox[0]), int(license_plate_bbox[1] - 10)),
                    font, font_scale, (0, 255, 255), font_thickness, cv2.LINE_AA)

    # Write the frame to the output video
    out.write(frame)

cap.release()
out.release()



0: 384x640 21 cars, 1 bus, 2 trucks, 337.0ms
Speed: 11.0ms preprocess, 337.0ms inference, 43.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 License_Plates, 264.0ms
Speed: 8.0ms preprocess, 264.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 cars, 1 bus, 2 trucks, 368.1ms
Speed: 9.0ms preprocess, 368.1ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 License_Plates, 267.0ms
Speed: 8.0ms preprocess, 267.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 2 trucks, 330.0ms
Speed: 7.0ms preprocess, 330.0ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 License_Plates, 319.0ms
Speed: 6.0ms preprocess, 319.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 cars, 1 bus, 2 trucks, 280.0ms
Speed: 7.0ms preprocess, 280.0ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 